In [1]:
import pyspark
from pyspark.sql.functions import (format_number, mean, max, 
                                   min, corr, year, month)

In [2]:
#Create a Spark Session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('solution').getOrCreate()

In [3]:
#Load Walmart stock data
df = sqlContext.sql("SELECT * FROM walmart")

In [4]:
#Briefly explore data
df.columns #column names

Out[ 6 ]: ['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Adj Close']

In [5]:
df.printSchema() #schema

root
-- Date: timestamp (nullable = true)
-- Open: double (nullable = true)
-- High: double (nullable = true)
-- Low: double (nullable = true)
-- Close: double (nullable = true)
-- Volume: integer (nullable = true)
-- Adj Close: double (nullable = true)

In [6]:
for row in df.head(5):
  print(row) #first five columns

Row(Date=datetime.datetime(2012, 1, 3, 0, 0), Open=59.970001, High=61.060001, Low=59.869999, Close=60.330002, Volume=12668800, Adj Close=52.619234999999996)
Row(Date=datetime.datetime(2012, 1, 4, 0, 0), Open=60.209998999999996, High=60.349998, Low=59.470001, Close=59.709998999999996, Volume=9593300, Adj Close=52.078475)
Row(Date=datetime.datetime(2012, 1, 5, 0, 0), Open=59.349998, High=59.619999, Low=58.369999, Close=59.419998, Volume=12768200, Adj Close=51.825539)
Row(Date=datetime.datetime(2012, 1, 6, 0, 0), Open=59.419998, High=59.450001, Low=58.869999, Close=59.0, Volume=8069400, Adj Close=51.45922)
Row(Date=datetime.datetime(2012, 1, 9, 0, 0), Open=59.029999, High=59.549999, Low=58.919998, Close=59.18, Volume=6679300, Adj Close=51.616215000000004)

In [7]:
df.describe().show() #summary statistics

+-------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+
summary| Open| High| Low| Close| Volume| Adj Close|
+-------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+
 count| 1258| 1258| 1258| 1258| 1258| 1258|
 mean| 72.35785375357709|72.83938807631165| 71.9186009594594|72.38844998012726|8222093.481717011|67.23883848728146|
 stddev| 6.76809024470826|6.768186808159218|6.744075756255496|6.756859163732991| 4519780.8431556|6.722609449996857|
 min|56.389998999999996| 57.060001| 56.299999| 56.419998| 2094900| 50.363689|
 max| 90.800003| 90.970001| 89.25| 90.470001| 80898100|84.91421600000001|
+-------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+

In [8]:
#Round numeric data
result = df.describe()
result.select(result['summary'],
             format_number(result['Open'].cast('float'),2).alias('Open'),
             format_number(result['High'].cast('float'),2).alias('High'),
             format_number(result['Low'].cast('float'),2).alias('Low'),
             format_number(result['Close'].cast('float'),2).alias('Close'),
             result['Volume'].cast('int').alias('Volume')
             ).show()

+-------+--------+--------+--------+--------+--------+
summary| Open| High| Low| Close| Volume|
+-------+--------+--------+--------+--------+--------+
 count|1,258.00|1,258.00|1,258.00|1,258.00| 1258|
 mean| 72.36| 72.84| 71.92| 72.39| 8222093|
 stddev| 6.77| 6.77| 6.74| 6.76| 4519780|
 min| 56.39| 57.06| 56.30| 56.42| 2094900|
 max| 90.80| 90.97| 89.25| 90.47|80898100|
+-------+--------+--------+--------+--------+--------+

In [9]:
#Calculate HV Ratio
df2 = df.withColumn("HV Ratio", df['High']/df['Volume'])
df2.select('HV Ratio').show()

In [10]:
#Day of peak price
df.orderBy(df['High'].desc()).head(1)[0][0]

Out[ 18 ]: datetime.datetime(2015, 1, 13, 0, 0)

In [11]:
#Mean of Close field
df.select(mean("Close")).withColumnRenamed('avg(Close)','Average Close').show()

+-----------------+
 Average Close|
+-----------------+
72.38844998012726|
+-----------------+

In [12]:
#Max and min of Volume field
df.select(min('Volume').alias('Minimum'),max('Volume').alias('Maximum')).show()

+-------+--------+
Minimum| Maximum|
+-------+--------+
2094900|80898100|
+-------+--------+

In [13]:
#Number of days with Close lower than $60
df.filter('Close < 60').count()
#Alternative: df.filter(df['Close'] < 60).count() 

Out[ 32 ]: 81

In [14]:
#What % of time was the High greater than $80?
(df.filter(df['High']>80).count() / df.count()) *100

Out[ 41 ]: 9.141494435612083

In [15]:
#Pearson correlation between High and Volume
df.select(corr('High','Volume')).show()

+-------------------+
 corr(High, Volume)|
+-------------------+
-0.3384326061737161|
+-------------------+

In [16]:
#What is the max High per year
yeardf = df.withColumn('Year',year(df['Date'])) #Format year column
max_df = yeardf.groupBy('Year').max()
max_df.select('Year', 'max(High)').orderBy('Year').show()



+----+---------+
Year|max(High)|
+----+---------+
2012|77.599998|
2013|81.370003|
2014|88.089996|
2015|90.970001|
2016|75.190002|
+----+---------+

In [17]:
#Average Close for Each Calendar Month
monthdf = df.withColumn('Month', month(df['Date']))
monthmeans = monthdf.select(['Month','Close']).groupBy('Month').mean()
monthmeans.select(['Month','avg(Close)']).orderBy('Month').show()

+-----+-----------------+
Month| avg(Close)|
+-----+-----------------+
 1|71.44801958415842|
 2| 71.306804443299|
 3|71.77794377570092|
 4|72.97361900952382|
 5|72.30971688679247|
 6| 72.4953774245283|
 7|74.43971943925233|
 8|73.02981855454546|
 9|72.18411785294116|
 10|71.57854545454543|
 11| 72.1110893069307|
 12|72.84792478301885|
+-----+-----------------+